In [80]:
#from bs4 import BeautifulSoup 
#import urllib
#import re
import string
import sys
import csv
import zipfile
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
import sklearn.datasets
import sklearn.cross_validation
import sklearn.decomposition
import sklearn.grid_search
import sklearn.neighbors
import sklearn.metrics
import sklearn.ensemble
import sklearn.tree

In [21]:
def get_zip_URL(url):
    r = requests.get(zipFileName).content
    s = StringIO.StringIO(r)
    zf = zipfile.ZipFile(s, 'r') # Read in a list of zipped files
    return zf

In [22]:
def get_zip_FS(filename):
    r = open(filename, 'rb')
    zf = zipfile.ZipFile(r, 'r') # Read in a list of zipped files
    return zf

In [23]:
infile = get_zip_FS('train.csv.zip')
df = pd.read_csv(infile.open(infile.namelist()[0]), index_col=0)
X = df.ix[:, range(0,len(df.columns)-1)]
Y = df.target

In [24]:
#infile = get_zip_FS('test.csv.zip')
#df = pd.read_csv(infile.open(infile.namelist()[0]), index_col=0)
#X_test = df.ix[:, range(0,len(df.columns))]

In [25]:
X_train, X_test, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    X, Y, test_size=0.33, random_state=42)

#print X_train.shape
#print X_test.shape
#print Y_train.shape
#print Y_test.shape

In [26]:
#def fix_result(in_result):
#    out_result = []
#    for i in in_result:
#        out_result.append(i)
#    return out_result

In [27]:
#def convert_result(in_result):
#    out_result = []
#    for i in in_result:
#        out_result.append(int(i[-1]))
#    return out_result

In [28]:
def match_submission_format(in_result):
    out_result = []
    j = 1
    for i in in_result:
        a = [0] * 10
        a[0] = j
        a[int(i[-1])] = 1
        out_result.append(a)
        j = j + 1
    return out_result

In [29]:
def write_submission_result(in_result):
    with open('submission.csv', 'wb') as of:
        csv_file = csv.writer(of, delimiter = ',')
        csv_file.writerow(['id','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
        for i in in_result:
            csv_file.writerow(i)

In [30]:
# random forest

#rfc.fit(X_train, Y_train)
#y_result = rfc.predict(X_test)

In [31]:
#truth = Y_test.astype('S')
#clf_result = y_result.astype('S')

In [32]:
#sklearn.metrics.accuracy_score(truth, clf_result)

In [33]:
# Random Forest
rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
sklearn.cross_validation.cross_val_score(rfc, X, Y, cv=10)

array([ 0.80604005,  0.80923922,  0.81276252,  0.80662359,  0.81108597,
        0.8081151 ,  0.80989331,  0.81781442,  0.80989331,  0.81449135])

In [34]:
#submission = match_submission_format(y_result)

In [35]:
#decision tree
#one_hot_encoding = pd.get_dummies(Y_train)
#clf = sklearn.tree.DecisionTreeClassifier(random_state=0)
#sklearn.cross_validation.cross_val_score(clf, X_train, one_hot_encoding, cv=10)

In [36]:
#write_submission_result(submission)

In [37]:
# perform svd on X first
#svd = sklearn.decomposition.TruncatedSVD(n_components=10)
#X_10d = svd.fit_transform(X)

In [19]:
# Random Forest
#rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
#np.mean(sklearn.cross_validation.cross_val_score(rfc, X_10d, Y, cv=10))

In [95]:
def clf_svd(X, Y, clf, dimension_reduction, no_of_feature=None):
    # perform dimension reduction on X, and run
    # classifer, return cross validation score matrix
    
    if (no_of_feature==None):
        no_of_feature = len(X.columns) 
        
    options = {'pca': sklearn.decomposition.PCA,
               't_svd': sklearn.decomposition.TruncatedSVD}
    
    # number of features must be greater than or equal to 2
    if len(X.columns) < 2:
        return False
    else:    
        result = {}
        for i in range(2, no_of_feature+1):
            # if number of features is same as i, no need to perform dimension reduction
            if (i == len(X.columns)):
                result[i] = sklearn.cross_validation.cross_val_score(clf, X, Y, cv=10)
            else:                
                svd = options[dimension_reduction](n_components=i)
                X_ds = svd.fit_transform(X)
                result[i] = sklearn.cross_validation.cross_val_score(clf, X_ds, Y, cv=10)
            
    return result

In [ ]:
result = clf_svd(X, Y, sklearn.ensemble.RandomForestClassifier(n_estimators=10), 'pca', 20)

In [74]:
totalresult = {}
for i in result.keys():
    totalresult[i] = np.mean(result[i])

In [82]:
max(totalresult.iteritems(), key=operator.itemgetter(1))

(44, 0.76172581815573615)

In [84]:
result = clf_svd(X, Y, sklearn.ensemble.RandomForestClassifier(n_estimators=10))

In [85]:
for i in result.keys():
    totalresult[i] = np.mean(result[i])

In [86]:
max(totalresult.iteritems(), key=operator.itemgetter(1))

(93, 0.78095686255325758)

In [87]:
totalresult

{2: 0.38789421899596599,
 3: 0.54961538467806625,
 4: 0.58529788962283702,
 5: 0.61462977760965432,
 6: 0.64666137826852188,
 7: 0.65642268020633909,
 8: 0.6785468078103688,
 9: 0.69386633036139655,
 10: 0.70125165609253282,
 11: 0.71005989107502077,
 12: 0.71720132364820144,
 13: 0.72318224061317271,
 14: 0.72911229740348271,
 15: 0.73303938420212233,
 16: 0.73627288589119133,
 17: 0.74210621571149249,
 18: 0.74066769953653844,
 19: 0.74247794996150041,
 20: 0.74318932766277168,
 21: 0.74483747086027408,
 22: 0.74535430022904581,
 23: 0.74579100228513118,
 24: 0.74637229825708462,
 25: 0.75012277824035212,
 26: 0.75030024950421925,
 27: 0.75089819112775935,
 28: 0.75201274909945615,
 29: 0.75354845954377958,
 30: 0.75372628549325449,
 31: 0.75445363545431698,
 32: 0.75576253979023289,
 33: 0.75545487737503347,
 34: 0.75831587176037674,
 35: 0.75766865243328962,
 36: 0.75904317204140603,
 37: 0.76036723134559736,
 38: 0.7606275644293008,
 39: 0.76048184275358266,
 40: 0.761515713096230

In [88]:
a = sklearn.cross_validation.cross_val_score(sklearn.ensemble.RandomForestClassifier(n_estimators=100), X, Y, cv=10)

In [89]:
a

array([ 0.80490956,  0.8077855 ,  0.81437803,  0.80355412,  0.80591467,
        0.80795344,  0.81247979,  0.81635952,  0.81021662,  0.81594695])